In [1]:
import torch
import torchaudio

In [2]:
torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(torch.__version__)
print(torchaudio.__version__)
print(device)

1.12.0
0.12.0
cuda


In [3]:
bundle = torchaudio.pipelines.WAV2VEC2_BASE

print("Sample Rate:", bundle.sample_rate)

model = bundle.get_model().to(device)

print(model.__class__)

Sample Rate: 16000
<class 'torchaudio.models.wav2vec2.model.Wav2Vec2Model'>


In [7]:
SPEECH_FILE = "/home/riaash/Documents/ACES lab/hifi-gan/test_files/LJ001-0001.wav"

waveform, sample_rate = torchaudio.load(SPEECH_FILE)
waveform = waveform.to(device)

In [8]:
if sample_rate != bundle.sample_rate:
    waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

In [9]:
with torch.inference_mode():
    features, _ = model.extract_features(waveform)

In [14]:
res, _ = model(waveform)

In [16]:
max(waveform[0])

tensor(0.8551, device='cuda:0')